# 4. トレーニングインフラ

In [ ]:
### 4.1 評価メトリクス定義
def compute_metrics(eval_preds):
    logits, labels = eval_preds  # Get logits and labels from eval_preds
    
    # Relax size limit for evaluation dataset
    max_samples = 100
    
    # Improve decoding process
    with torch.no_grad():
        logits = torch.tensor(logits).cpu()
        predictions = torch.argmax(logits, dim=-1)
        
        # Decode batch
        decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
        
        # Add more detailed logging
        logging.info(f"Sample prediction: {decoded_preds[0][:100]}...")
        
        del logits, predictions  # Memory release
        torch.cuda.empty_cache()
        
        # Define sentence ending patterns more flexibly
        sentence_end_patterns = {
            'question_patterns': [
                'かね', 'だろうか', 'ではないか',
                'のか', 'と思わないか', '考えてみよう',
            ],
            'statement_patterns': [
                'だね', 'なるほど', '興味深い',
                'といえよう', 'というべきだ'
            ],
            'reflection_patterns': [
                'かもしれない', 'のではないか',
                'と考えられる', 'といえそうだ'
            ]
        }
        
        # Auxiliary verb patterns
        auxiliary_patterns = [
            'である', 'だ', 'です', 'ます',
            'のだ', 'のです', 'のである'
        ]
        
        def calculate_style_consistency(text):
            sentences = text.split('。')
            if not sentences:
                return 0.0
                
            # Evaluate sentence ending style consistency
            end_style_scores = []
            for sent in sentences:
                if not sent.strip():
                    continue
                    
                # Evaluate sentence ending patterns (partial match allowed)
                pattern_found = False
                for pattern_type, patterns in sentence_end_patterns.items():
                    if any(p in sent[-10:] for p in patterns):  # Search within 10 characters at the end
                        pattern_found = True
                        break
                end_style_scores.append(1.0 if pattern_found else 0.0)
            
            # Evaluate auxiliary verb consistency
            aux_style_scores = []
            for sent in sentences:
                if not sent.strip():
                    continue
                    
                # Evaluate auxiliary verb usage in the sentence
                aux_found = any(p in sent for p in auxiliary_patterns)
                aux_style_scores.append(1.0 if aux_found else 0.0)
            
            # Evaluate sentence length consistency
            lengths = [len(s.strip()) for s in sentences if s.strip()]
            length_variance = np.var(lengths) if lengths else 0
            length_score = 1.0 / (1.0 + length_variance/100)  # Higher score if variance is small
            
            # Overall evaluation
            end_style_avg = np.mean(end_style_scores) if end_style_scores else 0
            aux_style_avg = np.mean(aux_style_scores) if aux_style_scores else 0
            
            # Weighting
            weights = {
                'end_style': 0.5,
                'aux_style': 0.3,
                'length_consistency': 0.2
            }
            
            return (
                weights['end_style'] * end_style_avg +
                weights['aux_style'] * aux_style_avg +
                weights['length_consistency'] * length_score
            )
        
        # Evaluate style consistency for each prediction
        style_scores = [calculate_style_consistency(pred) for pred in decoded_preds]
        
        # Evaluate dialogue flow
        def calculate_dialogue_flow(text):
            sentences = text.split('。')
            if not sentences:
                return 0.0
            
            # 質問文判定の改善
            question_markers = {
                'explicit': ['？', '?'],  # 明示的な質問符号
                'patterns': [
                    'かね', 'だろうか', 'ではないか', 'のか', 
                    'と思わないか', '考えてみよう',
                    'どう', 'いかが', 'なぜ', 'どのように',
                    '問', '教えて', '聞かせて'
                ]
            }
            
            def is_question(sentence):
                # 明示的な質問符号のチェック
                if any(marker in sentence for marker in question_markers['explicit']):
                    return True
                # 質問パターンのチェック
                if any(pattern in sentence for pattern in question_markers['patterns']):
                    return True
                return False
            
            # 各文を評価
            questions = sum(1 for s in sentences if is_question(s))
            total_sentences = len([s for s in sentences if s.strip()])
            ratio = questions / total_sentences if total_sentences else 0
            
            # 理想の比率(0.3)からの距離に基づいてスコアを計算
            balance_score = max(0.0, 1.0 - min(abs(0.3 - ratio), 0.2) * 2)
            
            # 2. Sentence length change
            lengths = [len(s.strip()) for s in sentences if s.strip()]
            length_variance = np.var(lengths) if len(lengths) > 1 else 0
            length_score = 1.0 / (1.0 + length_variance/500)  # Higher score if variance is small
            
            # 3. Use of conjunctions
            conjunctions = ['しかし', 'だから', 'また', 'そして', 'したがって']
            conj_count = sum(1 for s in sentences if any(c in s for c in conjunctions))
            conj_ratio = conj_count / len(sentences)
            conj_score = min(1.0, conj_ratio * 2)  # Evaluate moderate usage
            
            # Weighted average of scores
            weights = [0.5, 0.25, 0.25]  # Balance, length, conjunction weights
            final_score = sum(s * w for s, w in zip([balance_score, length_score, conj_score], weights))
            
            return max(0.1, min(1.0, final_score))  # Limit to range 0.1 to 1.0
        
        flow_scores = [calculate_dialogue_flow(pred) for pred in decoded_preds]
        
        style_score = np.mean(style_scores)
        flow_score = np.mean(flow_scores)
        
        # Add overall evaluation score
        combined_score = (style_score * 0.6 + flow_score * 0.4)  # Increase flow_score weight
        
        return {
            'style_consistency': style_score,
            'dialogue_flow': flow_score,
            'combined_score': combined_score
        }



<style>
pre {
    border: 1px solid #333;
    padding: 20px;
    margin: 20px 0;
    background-color: #000000;
    color: #d4d4d4;
    border-radius: 8px;
}
pre code {
    color: #d4d4d4;
    display: block;
    padding-bottom: 8px;
    background-color: #000000; 
}

.hljs, .language-python {
    background-color: #000000 !important;
}
</style>

<div style="background-color: #F9F4F0; padding: 10px; border-left: 5px solid #4CAF50; margin: 10px; width: 95%;">
    <details>
        <summary style="color: #8A6F5C; font-size: 1.17em; font-weight: bold;">claude解説</summary>
        <div style="color: #8A6F5C;">

このコードは、ソクラテス式の対話スタイルがどれだけ上手く実現できているかを評価するための仕組みを定義しています。以下に分かりやすく説明します：

## 1. 全体の目的
このコードは、モデルが生成した文章を評価して、以下の3つのスコアを計算します：
- スタイルの一貫性（style_consistency）
- 対話の流れ（dialogue_flow）
- 総合評価（combined_score）

## 2. スタイルの一貫性の評価
### 評価する3つのポイント：

1. **文末表現のパターン**
```python
sentence_end_patterns = {
    'question_patterns': ['かね', 'だろうか', 'ではないか'...],
    'statement_patterns': ['だね', 'なるほど', '興味深い'...],
    'reflection_patterns': ['かもしれない', 'のではないか'...]
}
```
例：
- 良い例：「その考えはどうだろうか」「なるほど、興味深い視点ですね」
- 悪い例：「はい」「いいえ」（ソクラテス式らしくない単純な返答）

2. **助動詞の使用**
```python
auxiliary_patterns = ['である', 'だ', 'です', 'ます'...]
```
例：
- 良い例：「それは本当である」「そう考えられるのです」
- 悪い例：「うん」「そう」（口語的すぎる表現）

3. **文の長さの一貫性**
- 文の長さがバラバラすぎないかをチェック
例：
- 良い例：「それはなぜだろうか。その理由について考えてみましょう。」（バランスの取れた文長）
- 悪い例：「なぜ？」「それについて詳しく説明すると、哲学的な観点から見た場合、様々な解釈が可能となり...」（極端に短い文と長い文が混在）

## 3. 対話の流れの評価
### 評価する3つのポイント：

1. **質問と説明のバランス**
- 理想的な質問の割合は30%程度
例：
- 良い例：「その点について深く考えてみましょう。なぜそう考えるのでしょうか？」
- 悪い例：「それはなぜですか？どう思いますか？どうしてそう考えましたか？」（質問が多すぎる）

2. **文の長さの変化**
- 文の長さに極端な変化がないかチェック

3. **接続詞の使用**
```python
conjunctions = ['しかし', 'だから', 'また', 'そして', 'したがって']
```
例：
- 良い例：「なるほど。しかし、その考えには別の視点もありそうですね。」
- 悪い例：「そうですね。そうですね。そうですね。」（接続詞がなく単調）

## 4. 最終スコアの計算
```python
combined_score = (style_score * 0.6 + flow_score * 0.4)
```
- スタイルの一貫性（60%）と対話の流れ（40%）を組み合わせて最終的な評価を行います
- スコアは0.0〜1.0の範囲で、1.0が最高評価です

このように、ソクラテス式の対話らしさを数値化して評価することで、モデルの学習がうまくいっているかを判断できます。

        
</div>
    </details>
</div>


In [ ]:

### 4.2 メモリ管理とモニタリング
def log_memory_usage():
    import psutil
    import torch
    
    # CPU memory
    process = psutil.Process()
    cpu_memory = process.memory_info().rss / 1024 / 1024  # MB
    
    # GPU memory
    gpu_memory = []
    if torch.cuda.is_available():
        for i in range(torch.cuda.device_count()):
            gpu_memory.append({
                'device': i,
                'allocated': torch.cuda.memory_allocated(i) / 1024 / 1024,  # MB
                'reserved': torch.cuda.memory_reserved(i) / 1024 / 1024,    # MB
                'max_allocated': torch.cuda.max_memory_allocated(i) / 1024 / 1024  # MB
            })
    
    logging.info(f"CPU Memory usage: {cpu_memory:.2f} MB")
    for gpu in gpu_memory:
        logging.info(f"GPU {gpu['device']} Memory:")
        logging.info(f"  - Allocated: {gpu['allocated']:.2f} MB")
        logging.info(f"  - Reserved: {gpu['reserved']:.2f} MB")
        logging.info(f"  - Max Allocated: {gpu['max_allocated']:.2f} MB")

# Log dataset size
logging.info(f"Total dataset size: {len(dataset)}")
log_memory_usage()

def clear_memory():
    gc.collect()
    torch.cuda.empty_cache()



<style>
pre {
    border: 1px solid #333;
    padding: 20px;
    margin: 20px 0;
    background-color: #000000;
    color: #d4d4d4;
    border-radius: 8px;
}
pre code {
    color: #d4d4d4;
    display: block;
    padding-bottom: 8px;
    background-color: #000000; 
}

.hljs, .language-python {
    background-color: #000000 !important;
}
</style>

<div style="background-color: #F9F4F0; padding: 10px; border-left: 5px solid #4CAF50; margin: 10px; width: 95%;">
    <details>
        <summary style="color: #8A6F5C; font-size: 1.17em; font-weight: bold;">claude解説</summary>
        <div style="color: #8A6F5C;">

このコードはメモリの使用状況を監視・管理するための機能を実装しています。以下に分かりやすく説明します：

## 1. メモリ使用状況のログ記録（log_memory_usage関数）

### CPUメモリの監視
```python
process = psutil.Process()
cpu_memory = process.memory_info().rss / 1024 / 1024  # MB
```
- プログラムが使用しているCPUメモリ量を測定
- MBに変換して記録（例：「CPU Memory usage: 4562.34 MB」）

### GPUメモリの監視
```python
gpu_memory.append({
    'device': i,
    'allocated': torch.cuda.memory_allocated(i) / 1024 / 1024,  # MB
    'reserved': torch.cuda.memory_reserved(i) / 1024 / 1024,    # MB
    'max_allocated': torch.cuda.max_memory_allocated(i) / 1024 / 1024  # MB
})
```

3種類のGPUメモリ情報を記録：
1. **allocated（割り当て済み）**: 現在実際に使用中のメモリ
   - 例：「GPU 0 Memory: Allocated: 2048.00 MB」

2. **reserved（予約済み）**: 確保されているが未使用のメモリ
   - 例：「GPU 0 Memory: Reserved: 3072.00 MB」

3. **max_allocated（最大割り当て）**: プログラム開始からの最大使用量
   - 例：「GPU 0 Memory: Max Allocated: 4096.00 MB」

## 2. メモリのクリーンアップ（clear_memory関数）

```python
def clear_memory():
    gc.collect()           # Pythonのガベージコレクションを実行
    torch.cuda.empty_cache() # GPUのキャッシュをクリア
```

このクリーンアップが重要な理由：
- ソクラテス式対話モデルの学習では大量のメモリを使用
- メモリリークを防ぎ、長時間の学習を安定して行える
- 例：長い対話履歴を処理した後のメモリ解放

## 3. 使用例と重要性

### データセットサイズの記録
```python
logging.info(f"Total dataset size: {len(dataset)}")
```
- 例：「Total dataset size: 10000」

### メモリ監視の実際の使用シーン：
1. **学習開始前**：
   - 利用可能なメモリを確認
   - 適切なバッチサイズの決定に使用

2. **学習中**：
   - メモリリークの検出
   - 異常な使用量の早期発見

3. **エラー発生時**：
   - メモリ不足が原因かどうかの診断
   - 適切な対処方法の決定

### 具体例：
```python
# 学習の各ステップで
log_memory_usage()  # メモリ使用状況を確認
if 大きな対話データの処理後:
    clear_memory()  # メモリをクリーンアップ
```

このモニタリングにより：
- 安定した学習環境の維持
- メモリ関連の問題を早期発見
- システムリソースの効率的な使用
が可能になります。

        
</div>
    </details>
</div>


In [ ]:
### 4.3 コールバック実装
class StyleCallback(TrainerCallback):
    def __init__(self):
        self.style_scores = []
        self.flow_scores = []
        
    def on_evaluate(self, args, state, control, metrics, **kwargs):
        if 'eval_style_consistency' in metrics:
            self.style_scores.append(metrics['eval_style_consistency'])
            self.flow_scores.append(metrics['eval_dialogue_flow'])
            
            # Log detailed information
            logging.info(f"Step {state.global_step}:")
            logging.info(f"Style Consistency: {metrics['eval_style_consistency']:.3f}")
            logging.info(f"Dialogue Flow: {metrics['eval_dialogue_flow']:.3f}")
    
    def on_train_end(self, args, state, control, **kwargs):
        # Log overall evaluation
        avg_style = sum(self.style_scores) / len(self.style_scores) if self.style_scores else 0
        avg_flow = sum(self.flow_scores) / len(self.flow_scores) if self.flow_scores else 0
        
        logging.info("Training Complete!")
        logging.info(f"Average Style Consistency: {avg_style:.3f}")
        logging.info(f"Average Dialogue Flow: {avg_flow:.3f}")

# Extend custom callbacks

class TrainingMonitorCallback(TrainerCallback):
    def __init__(self):
        # Import psutil here as well for safety
        import psutil
        self.train_start_time = None
        self.metrics_history = {
            'step': [],
            'style_consistency': [],
            'dialogue_flow': [],
            'combined_score': [],
            'loss': [],
            'learning_rate': [],
            'epoch': [],
            'cpu_ram_usage': [],
            'gpu_vram_usage': [],
            'gpu_utilization': [],
            'batch_size': [],
            'moving_avg_loss': [],
            # 新しい詳細メトリクス
            'lr_schedule': [],
            'batch_metrics': [],
            'gpu_metrics': [],
            'grad_norm': []
        }
        self.peak_metrics = {
            'cpu_ram': 0,
            'gpu_vram': 0,
            'gpu_util': 0
        }
        self.output_dir = Path(f"{BASE_OUTPUT_DIR}/training_progress")
        self.output_dir.mkdir(parents=True, exist_ok=True)
        
    def _record_resource_usage(self):
        """Record current resource usage with timestamp"""
        import psutil
        import torch
        current_time = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        
        # CPU RAM
        cpu_ram = psutil.Process().memory_info().rss / (1024 * 1024 * 1024)  # GB
        self.peak_metrics['cpu_ram'] = max(self.peak_metrics['cpu_ram'], cpu_ram)
        
        # GPU metrics with timestamp
        if torch.cuda.is_available():
            gpu_metrics = []
            for i in range(torch.cuda.device_count()):
                vram_used = torch.cuda.memory_allocated(i) / (1024 * 1024 * 1024)  # GB
                self.peak_metrics['gpu_vram'] = max(self.peak_metrics['gpu_vram'], vram_used)
                
                # GPU utilization (requires nvidia-smi)
                try:
                    import subprocess
                    result = subprocess.check_output(['nvidia-smi', '--query-gpu=utilization.gpu', '--format=csv,noheader,nounits'])
                    gpu_util = float(result.decode('utf-8').strip())
                    self.peak_metrics['gpu_util'] = max(self.peak_metrics['gpu_util'], gpu_util)
                except:
                    gpu_util = 0
                
                gpu_metrics.append({
                    'device': i,
                    'vram_used': vram_used,
                    'utilization': gpu_util
                })
                
            # 時系列データとして保存
            self.metrics_history['gpu_metrics'].append({
                'timestamp': current_time,
                'metrics': gpu_metrics
            })
                
            self.metrics_history['cpu_ram_usage'].append(cpu_ram)
            self.metrics_history['gpu_vram_usage'].append(vram_used)
            self.metrics_history['gpu_utilization'].append(gpu_util)
    
    def on_train_begin(self, args, state, control, **kwargs):
        self.train_start_time = datetime.now()
        logging.info("Training started at: %s", self.train_start_time)
        self._record_resource_usage()
        
    def on_log(self, args, state, control, logs=None, **kwargs):
        if logs:
            current_time = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
            
            # 学習率とスケジューリングの記録
            if 'learning_rate' in logs:
                self.metrics_history['lr_schedule'].append({
                    'timestamp': current_time,
                    'step': state.global_step,
                    'learning_rate': logs['learning_rate'],
                    'schedule_type': args.lr_scheduler_type
                })
                self.metrics_history['learning_rate'].append(logs['learning_rate'])
            
            # バッチサイズと損失値の関連を記録
            if 'loss' in logs:
                self.metrics_history['batch_metrics'].append({
                    'timestamp': current_time,
                    'step': state.global_step,
                    'batch_size': args.per_device_train_batch_size,
                    'loss': logs['loss'],
                    'grad_norm': logs.get('grad_norm', None)
                })
                self.metrics_history['loss'].append(logs['loss'])
                self.metrics_history['batch_size'].append(args.per_device_train_batch_size)
                if 'grad_norm' in logs:
                    self.metrics_history['grad_norm'].append(logs['grad_norm'])
            
            # 移動平均の計算と記録
            if len(self.metrics_history['loss']) > 10:
                avg_loss = sum(self.metrics_history['loss'][-10:]) / 10
                self.metrics_history['moving_avg_loss'].append(avg_loss)
                logging.info(f"Moving average loss (last 10 steps): {avg_loss:.4f}")
            
            logging.info(f"Step {state.global_step}: {logs}")
            if 'grad_norm' in logs:
                logging.info(f"Gradient norm: {logs['grad_norm']:.4f}")
            
        self._record_resource_usage()
        
    def on_train_end(self, args, state, control, **kwargs):
        training_duration = datetime.now() - self.train_start_time
        
        # 詳細な学習履歴の保存
        training_history = {
            'lr_schedule': self.metrics_history['lr_schedule'],
            'batch_metrics': self.metrics_history['batch_metrics'],
            'gpu_metrics': self.metrics_history['gpu_metrics'],
            'moving_avg_loss': self.metrics_history['moving_avg_loss']
        }
        
        # 学習履歴をJSONファイルとして保存
        history_file = self.output_dir / 'training_history.json'
        with open(history_file, 'w', encoding='utf-8') as f:
            json.dump(training_history, f, indent=2, ensure_ascii=False)
        
        # 基本的なメトリクスのログ出力
        logging.info(f"Training completed. Total duration: {training_duration}")
        logging.info(f"Peak CPU RAM usage: {self.peak_metrics['cpu_ram']:.2f} GB")
        logging.info(f"Peak GPU VRAM usage: {self.peak_metrics['gpu_vram']:.2f} GB")
        logging.info(f"Peak GPU utilization: {self.peak_metrics['gpu_util']:.1f}%")
        
        # 最終サマリーの作成と保存
        summary = {
            'training_duration': str(training_duration),
            'final_loss': self.metrics_history['loss'][-1] if self.metrics_history['loss'] else None,
            'best_combined_score': max(filter(None, self.metrics_history['combined_score'])) if self.metrics_history['combined_score'] else None,
            'total_steps': len(self.metrics_history['step']),
            'final_epoch': self.metrics_history['epoch'][-1] if self.metrics_history['epoch'] else None,
            'learning_rate_summary': {
                'initial': self.metrics_history['learning_rate'][0] if self.metrics_history['learning_rate'] else None,
                'final': self.metrics_history['learning_rate'][-1] if self.metrics_history['learning_rate'] else None,
                'schedule_type': args.lr_scheduler_type
            },
            'loss_summary': {
                'final_moving_avg': self.metrics_history['moving_avg_loss'][-1] if self.metrics_history['moving_avg_loss'] else None,
                'best_loss': min(self.metrics_history['loss']) if self.metrics_history['loss'] else None
            },
            'resource_usage': {
                'peak_cpu_ram_gb': self.peak_metrics['cpu_ram'],
                'peak_gpu_vram_gb': self.peak_metrics['gpu_vram'],
                'peak_gpu_utilization': self.peak_metrics['gpu_util']
            },
            'hardware_info': {
                'cpu_info': self._get_cpu_info(),
                'gpu_info': self._get_gpu_info(),
                'total_ram': self._get_total_ram()
            }
        }
        
        # サマリーをJSONファイルとして保存
        with open(self.output_dir / 'training_summary.json', 'w', encoding='utf-8') as f:
            json.dump(summary, f, indent=2, ensure_ascii=False)
            
        logging.info("Training Complete!")
        logging.info(f"Training duration: {summary['training_duration']}")
        
        # Noneチェックを追加
        if summary['loss_summary']['final_moving_avg'] is not None:
            logging.info(f"Final moving average loss: {summary['loss_summary']['final_moving_avg']:.4f}")
        if summary['loss_summary']['best_loss'] is not None:
            logging.info(f"Best loss achieved: {summary['loss_summary']['best_loss']:.4f}")
        
        logging.info(f"Peak CPU RAM usage: {summary['resource_usage']['peak_cpu_ram_gb']:.2f} GB")
        logging.info(f"Peak GPU VRAM usage: {summary['resource_usage']['peak_gpu_vram_gb']:.2f} GB")
        logging.info(f"Peak GPU utilization: {summary['resource_usage']['peak_gpu_utilization']:.1f}%")

    def _get_cpu_info(self):
        import cpuinfo
        try:
            info = cpuinfo.get_cpu_info()
            return {
                'model': info.get('brand_raw', 'Unknown'),
                'cores': psutil.cpu_count(logical=False),
                'threads': psutil.cpu_count(logical=True)
            }
        except:
            return "Failed to get CPU info"
            
    def _get_gpu_info(self):
        if not torch.cuda.is_available():
            return "No GPU available"
        try:
            import subprocess
            result = subprocess.check_output(['nvidia-smi', '--query-gpu=gpu_name,memory.total', '--format=csv,noheader,nounits'])
            gpus = result.decode('utf-8').strip().split('\n')
            return [{'model': g.split(',')[0], 'memory': float(g.split(',')[1])/1024} for g in gpus]
        except:
            return "Failed to get GPU info"
            
    def _get_total_ram(self):
        return psutil.virtual_memory().total / (1024 * 1024 * 1024)  # GB


<style>
pre {
    border: 1px solid #333;
    padding: 20px;
    margin: 20px 0;
    background-color: #000000;
    color: #d4d4d4;
    border-radius: 8px;
}
pre code {
    color: #d4d4d4;
    display: block;
    padding-bottom: 8px;
    background-color: #000000; 
}

.hljs, .language-python {
    background-color: #000000 !important;
}
</style>

<div style="background-color: #F9F4F0; padding: 10px; border-left: 5px solid #4CAF50; margin: 10px; width: 95%;">
    <details>
        <summary style="color: #8A6F5C; font-size: 1.17em; font-weight: bold;">claude解説</summary>
        <div style="color: #8A6F5C;">



このコードは学習の進行状況を監視・記録するためのコールバック（監視プログラム）を実装しています。2つの主要なクラスについて説明します：

## 1. StyleCallback クラス
ソクラテス式対話の品質を監視するためのクラスです。

### 主な機能：
```python
class StyleCallback(TrainerCallback):
    def __init__(self):
        self.style_scores = []  # スタイルスコアの履歴
        self.flow_scores = []   # 対話の流れスコアの履歴
```

### 評価時の処理：
```python
def on_evaluate(self, args, state, control, metrics, **kwargs):
    # スタイルと対話の流れのスコアを記録
    logging.info(f"Style Consistency: {metrics['eval_style_consistency']:.3f}")
    logging.info(f"Dialogue Flow: {metrics['eval_dialogue_flow']:.3f}")
```

例：
```
Step 100:
Style Consistency: 0.856
Dialogue Flow: 0.742
```

### 学習終了時の処理：
- 全期間の平均スコアを計算して表示
```
Training Complete!
Average Style Consistency: 0.823
Average Dialogue Flow: 0.751
```

## 2. TrainingMonitorCallback クラス
学習全体の進行状況を詳細に監視・記録するクラスです。

### 記録する主な情報：
```python
self.metrics_history = {
    'style_consistency': [],    # スタイルの一貫性
    'dialogue_flow': [],        # 対話の流れ
    'loss': [],                # 損失値
    'learning_rate': [],       # 学習率
    'cpu_ram_usage': [],       # CPUメモリ使用量
    'gpu_vram_usage': [],      # GPUメモリ使用量
    # その他の指標...
}
```

### 主な機能：

1. **学習開始時の記録**
```python
def on_train_begin(self, args, state, control, **kwargs):
    self.train_start_time = datetime.now()
    logging.info("Training started at: %s", self.train_start_time)
```

2. **定期的な学習状況の記録**
```python
def on_log(self, args, state, control, logs=None, **kwargs):
    # 学習率の記録
    # 損失値の記録
    # メモリ使用量の記録
```

例：
```
Step 100: 
Learning rate: 0.0001
Loss: 2.345
Moving average loss (last 10 steps): 2.456
```

3. **学習終了時のサマリー作成**
```python
summary = {
    'training_duration': '2時間30分',
    'final_loss': 1.234,
    'best_combined_score': 0.876,
    'resource_usage': {
        'peak_cpu_ram_gb': 16.5,
        'peak_gpu_vram_gb': 8.2,
        'peak_gpu_utilization': 95.4
    }
}
```

### 出力例：
```
Training Complete!
Training duration: 2:30:45
Final moving average loss: 1.234
Peak CPU RAM usage: 16.50 GB
Peak GPU VRAM usage: 8.20 GB
Peak GPU utilization: 95.4%
```

このように、学習の進行状況を多角的に監視・記録することで：
- モデルの品質改善の追跡
- リソース使用の最適化
- 問題の早期発見
が可能になります。

        
</div>
    </details>
</div>


In [ ]:
### 4.4 カスタムトレーナー定義
# Training step customization
class CustomTrainer(Trainer):
    def training_step(self, *args, **kwargs):
        loss = super().training_step(*args, **kwargs)
        if self.state.global_step % 50 == 0:
            clear_memory()
            gc.collect()
            torch.cuda.empty_cache()
        return loss

# Evaluation customization
class CustomTrainer(Trainer):
    def evaluate(self, eval_dataset=None, ignore_keys=None, metric_key_prefix="eval"):
        eval_dataset = eval_dataset if eval_dataset is not None else self.eval_dataset
        if eval_dataset is not None:
            # Limit evaluation dataset to 100 samples
            eval_dataset = eval_dataset.select(range(min(100, len(eval_dataset))))
        return super().evaluate(eval_dataset, ignore_keys, metric_key_prefix)

# Trainer initialization
trainer = CustomTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    callbacks=[StyleCallback(), TrainingMonitorCallback()],
)


<style>
pre {
    border: 1px solid #333;
    padding: 20px;
    margin: 20px 0;
    background-color: #000000;
    color: #d4d4d4;
    border-radius: 8px;
}
pre code {
    color: #d4d4d4;
    display: block;
    padding-bottom: 8px;
    background-color: #000000; 
}

.hljs, .language-python {
    background-color: #000000 !important;
}
</style>

<div style="background-color: #F9F4F0; padding: 10px; border-left: 5px solid #4CAF50; margin: 10px; width: 95%;">
    <details>
        <summary style="color: #8A6F5C; font-size: 1.17em; font-weight: bold;">claude解説</summary>
        <div style="color: #8A6F5C;">



このコードはカスタマイズされたトレーニング処理を定義しています。以下に分かりやすく説明します：

## 1. カスタムトレーナーの学習ステップ
```python
class CustomTrainer(Trainer):
    def training_step(self, *args, **kwargs):
        loss = super().training_step(*args, **kwargs)
        if self.state.global_step % 50 == 0:  # 50ステップごとに
            clear_memory()                     # メモリクリア
            gc.collect()                       # ガベージコレクション
            torch.cuda.empty_cache()           # GPUキャッシュクリア
        return loss
```

### 目的：
- メモリの効率的な管理
- 長時間の学習を安定して行う

### 具体例：
```python
ステップ 50: メモリクリア実行
- Before: GPU使用量 8GB
- After:  GPU使用量 6GB

ステップ 100: メモリクリア実行
- Before: GPU使用量 8.5GB
- After:  GPU使用量 6GB
```

## 2. カスタムトレーナーの評価機能
```python
class CustomTrainer(Trainer):
    def evaluate(self, eval_dataset=None, ignore_keys=None, metric_key_prefix="eval"):
        eval_dataset = eval_dataset if eval_dataset is not None else self.eval_dataset
        if eval_dataset is not None:
            # 評価用データセットを100サンプルに制限
            eval_dataset = eval_dataset.select(range(min(100, len(eval_dataset))))
        return super().evaluate(eval_dataset, ignore_keys, metric_key_prefix)
```

### 目的：
- 評価処理の効率化
- メモリ使用量の削減

### 具体例：
```python
元のデータセット: 1000対話
↓
評価用に制限: 100対話
- 「その考えはどうでしょうか？」
- 「なるほど、興味深い視点ですね」
- 「それはなぜだと思いますか？」
など、100の対話サンプルで評価
```

## 3. トレーナーの初期化
```python
trainer = CustomTrainer(
    model=model,                    # 学習対象のモデル
    args=training_args,            # 学習設定
    train_dataset=train_dataset,   # 学習用データセット
    eval_dataset=eval_dataset,     # 評価用データセット
    data_collator=data_collator,   # データ整形用関数
    compute_metrics=compute_metrics,# 評価指標計算
    callbacks=[
        StyleCallback(),           # スタイル評価用コールバック
        TrainingMonitorCallback()  # 学習監視用コールバック
    ],
)
```

### 設定の意味：
1. **model**: ソクラテス式対話を生成するモデル
2. **training_args**: バッチサイズ、学習率などの設定
3. **datasets**: 学習用と評価用の対話データ
4. **compute_metrics**: スタイルと対話の流れを評価
5. **callbacks**: 学習進捗の監視と記録

### 実際の使用例：
```python
# トレーナーの使用
trainer.train()

# 出力例：
Epoch 1/30:
- Loss: 2.345
- Style Consistency: 0.756
- Dialogue Flow: 0.678
- Memory Usage: 6.5GB

Epoch 2/30:
- Loss: 2.123
- Style Consistency: 0.789
- Dialogue Flow: 0.701
- Memory Usage: 6.5GB
```

このカスタムトレーナーにより：
- メモリ使用の最適化
- 効率的な評価処理
- 詳細な学習進捗の監視
が実現され、安定した学習が可能になります。

        
</div>
    </details>
</div>
